In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from scipy import stats
from statsmodels.stats import proportion

In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# import warnings 
# warnings.filterwarnings('ignore')

In [4]:
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
df = pd.read_excel('finalAll.xlsx')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9624 entries, 0 to 9623
Data columns (total 21 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   отзыв                                                                         9624 non-null   object 
 1   Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный)       9623 non-null   object 
 2   ответ психолога на отзыв                                                      5229 non-null   object 
 3   ссылка на профиль
                                                            9624 non-null   object 
 4   пол (м или ж)                                                                 9624 non-null   object 
 5   возраст                                                                       8355 non-null   float64
 6   опыт работы психологом (количест

In [7]:
df[df.columns[17]] = pd.to_numeric(df[df.columns[17]])

Найдём связь между ценой и отзывами

In [8]:
df['is_negative'] = df[df.columns[1]].apply(lambda x: 1 if x == 'о' else 0)

In [9]:
df.groupby(df.columns[3]).mean()[[df.columns[17], 'is_negative']].corr()

,стоимость одной очной консультации (в рублях),is_negative
стоимость одной очной консультации (в рублях),1.000000,0.125368
is_negative,0.125368,1.000000


In [10]:
k = df.groupby(df.columns[3]).mean()[[df.columns[17], 'is_negative']].dropna()

In [11]:
stats.pearsonr(k.iloc[:, 0], k.iloc[:, 1])

(0.1253681118342847, 0.03252838476045884)

In [12]:
df[df.columns[6]] = pd.to_numeric(df[df.columns[6]])

In [13]:
df[df.columns[5]] = pd.to_numeric(df[df.columns[5]])

In [14]:
df.groupby(df.columns[3]).mean()[[df.columns[11], 'is_negative']].corr()

,количество онлайн-консультаций (число),is_negative
количество онлайн-консультаций (число),1.000000,-0.017173
is_negative,-0.017173,1.000000


Так что нет корреляции между возрастом, опытом, кол-вом обратившихся с долей отрицательных отзывов

In [15]:
df.groupby(df.columns[3]).mean()[[df.columns[5], 'is_negative']].corr()

,возраст,is_negative
возраст,1.000000,0.038055
is_negative,0.038055,1.000000


In [16]:
df[df[df.columns[1]] == 'о'][df.columns[4]].value_counts()

ж    116
м     70
Name: пол (м или ж), dtype: int64

In [17]:
df[df.columns[4]].value_counts()

ж    6554
м    3070
Name: пол (м или ж), dtype: int64

Распределение психологов по полу

In [18]:
df.groupby([df.columns[3], df.columns[4]]).mean().reset_index()[df.columns[4]].value_counts()

ж    214
м     86
Name: пол (м или ж), dtype: int64

In [19]:
df.groupby([df.columns[3], df.columns[4]]).mean().reset_index()[df.columns[4]].value_counts(normalize=True).round(2)

ж    0.71
м    0.29
Name: пол (м или ж), dtype: float64

In [20]:
df.groupby([df.columns[3], df.columns[4]]).mean().reset_index()[df.columns[4]].shape

(300,)

In [21]:
df[df[df.columns[4]] == 'м']['is_negative'].value_counts(normalize=True).round(3)

0    0.977
1    0.023
Name: is_negative, dtype: float64

In [22]:
df[df[df.columns[4]] == 'ж']['is_negative'].value_counts(normalize=True).round(3)

0    0.982
1    0.018
Name: is_negative, dtype: float64

In [23]:
df[df[df.columns[4]] == 'м']['is_negative'].value_counts()

0    3000
1      70
Name: is_negative, dtype: int64

In [24]:
df[df[df.columns[4]] == 'ж']['is_negative'].value_counts()

0    6438
1     116
Name: is_negative, dtype: int64

In [25]:
df[df[df.columns[4]] == 'м']['is_negative'].value_counts(normalize=True)[0] - df[df[df.columns[4]] == 'м']['is_negative'].value_counts(normalize=True)[1]

0.9543973941368078

In [26]:
df[df[df.columns[4]] == 'ж']['is_negative'].value_counts(normalize=True)[0] - df[df[df.columns[4]] == 'ж']['is_negative'].value_counts(normalize=True)[1]

0.9646017699115044

In [27]:
proportion.proportions_ztest(np.array([3000, 6438]), np.array([3000 + 70, 6438 + 116]), alternative='two-sided')

(-1.6945726870632591, 0.09015651201912227)

In [28]:
count = np.array([3721, 5717])
nobs = np.array([3721 + 93, 5717 + 93])
stat, pval = proportion.proportions_ztest(count, nobs)
print('{0:0.3f}'.format(pval))

0.004


In [31]:
df.groupby([df.columns[3], df.columns[4]]).mean().reset_index().groupby(df.columns[4])

In [32]:
def count_ngrams(series: pd.Series, n: int) -> pd.Series:
    ngrams = series.copy().str.split(' ').explode()
    for i in range(1, n):
        ngrams += ' ' + ngrams.groupby(level=0).shift(-i)
        ngrams = ngrams.dropna()
    return ngrams.value_counts()    

In [33]:
(df[df.columns[1]] == 'о').value_counts()

False    9438
True      186
Name: Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный), dtype: int64

In [34]:
df[df.columns[1]].value_counts()

п    8878
н     559
о     186
Name: Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный), dtype: int64

In [35]:
(8878 + 559) / 186

50.736559139784944

In [36]:
# df.loc[df[df.columns[1]] == 'о', 'отзыв']

In [37]:
# print(count_ngrams(df.loc[df[df.columns[1]] == 'о', 'отзыв'], 2).index[:500])

In [38]:
# f = df['отзыв'][0]
# print(f)

как-то получше убрать знаки препинания, или вовсе переписать эту функцию.

In [39]:
def preprocess_text(texts):
    russian_stopwords = stopwords.words("russian")
    mystem = Mystem() 
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(texts, 1000)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' br ' for txt in txtp])

        words = mystem.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != ''\
            and txt not in russian_stopwords and all(i not in punctuation for i in txt) and len(txt) > 1 and not any(char.isdigit() for char in txt):
                if txt == 'br':
                    res.append(doc)
                    doc = []
                else:
                    doc.append(txt)
    return res

In [40]:
%%time
reviews_processed = preprocess_text(df['отзыв'])

Wall time: 37.2 s


Сделать то же самое, но со школа, с ценой, с какими-то ещё характеристиками. В каком случае люди оказываются недовольными чаще.

In [41]:
# preprocess_text(df[df.columns[-2]][:20])

In [42]:
def get_freq_words(sentiment, condition=None):
    blank = []
    if condition is not None:
        [blank.extend(i) for i in pd.Series(reviews_processed)[(df[df.columns[1]] == sentiment) & condition]]
    else:
        [blank.extend(i) for i in pd.Series(reviews_processed)[df[df.columns[1]] == sentiment]]
    return pd.Series(blank).value_counts(normalize=True)

In [43]:
pos = get_freq_words('п')
neg = get_freq_words('о')

In [44]:
pos_male = get_freq_words('п', df[df.columns[4]] == 'м')
pos_female = get_freq_words('п', df[df.columns[4]] == 'ж')

In [45]:
multi_freq_sex = {}
for i in pos_female.index:
    if i in pos_male.index:
        multi_freq_sex[i] = pos_female[i] / pos_male[i]
multi_freq_sex = pd.Series(multi_freq_sex).sort_values()

In [63]:
min_in_male = 10
min_multiplicator = 1.5
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for i in multi_freq_sex[:].index:
        if pos_male[i] / pos_male.min() >= min_in_male:
            print(i, (1 / multi_freq_sex[i]).round(4))
        if 1 / multi_freq_sex[i] < min_multiplicator:
            break

антон 461.9066
андрей 275.4045
евгений 243.5156
олег 179.7377
алексей 168.1417
валерьевич 164.2764
сергей 131.8076
виталий 110.1618
дмитрий 99.5322
евгения 75.3739
владимир 69.5759
павел 61.8452
роман 52.1819
александр 24.4176
ас 23.192
гипноз 21.2593
диплом 14.495
давний 13.5286
доктор 9.1111
невроз 7.8917
бессонница 7.2475
жалоба 6.4422
продвижение 6.3502
мужской 6.2812
диагноз 6.2275
мак 5.798
нестандартный 5.798
метафорический 5.798
симптом 5.5007
лекарство 5.4115
покидать 5.4115
потенциал 5.3148
волшебник 5.1538
па 4.6706
входить 4.5095
лечение 4.2519
окр 4.2519
создаваться 3.8653
уменьшаться 3.8653
панический 3.8116
атака 3.7086
групповой 3.6896
схема 3.6506
улыбаться 3.5892
программа 3.4512
страдание 3.4358
крутой 3.3822
антидепрессант 3.3131
обходиться 3.3131
пять 3.3131
привычный 3.3131
пациент 3.2855
врач 3.1625
назначать 3.1406
защита 3.1406
руководство 3.1137
пройти 3.037
свежий 3.037
заболевание 3.037
психотерапевт 2.9763
обучение 2.9733
психотерапия 2.9697
созависимость 2

In [62]:
min_in_male = 4
min_multiplicator = 1.5
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for i in multi_freq_sex[::-1].index:
        if pos_male[i] / pos_male.min() >= min_in_male:
            print(i, (multi_freq_sex[i]).round(2))
        if multi_freq_sex[i] < min_multiplicator:
            break

анна 54.46
ольга 47.29
александровна 14.83
елена 13.51
ирина 5.37
александра 4.4
налаживать 4.23
дочь 3.79
мечта 3.19
учитель 3.1
владимировна 2.97
подруга 2.91
ивановна 2.83
переосмыслять 2.73
праздник 2.67
сюда 2.59
сочувствие 2.48
одиночество 2.46
наталья 2.43
спрашивать 2.41
женщина 2.4
обо 2.38
прежде 2.37
переживать 2.35
разрыв 2.33
период 2.32
успешный 2.28
влиять 2.24
призвание 2.2
предел 2.2
грустно 2.2
твой 2.18
женский 2.18
мимо 2.17
расставлять 2.17
нормально 2.14
любовь 2.14
марафон 2.13
пересматривать 2.11
вскрывать 2.07
договариваться 2.07
еда 2.07
осуждение 2.07
сопереживание 2.07
мыслить 2.07
родной 2.07
хватать 2.07
невероятный 2.02
прислушиваться 2.02
дочка 2.0
здравствовать 2.0
болеть 1.98
рисунок 1.97
гармоничный 1.97
наверно 1.94
вдохновение 1.94
посылать 1.94
наслаждаться 1.94
заботливый 1.94
озвучивать 1.94
значительный 1.94
путешествие 1.93
великолепный 1.92
прощать 1.91
нелегкий 1.91
задумываться 1.91
мудрость 1.9
открываться 1.89
уверять 1.88
злость 1.88
вдох

In [48]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(pos[:20])

In [49]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(neg[:20])

In [50]:
multi_freq = {}
for i in neg.index:
    if i in pos.index:
        multi_freq[i] = pos[i] / neg[i]
multi_freq = pd.Series(multi_freq).sort_values()

In [60]:
min_in_neg = 4
min_multiplicator = 2
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for i in multi_freq[:].index:
        if neg[i] / neg.min() >= min_in_neg:
            print(i, (1 / multi_freq[i]).round(4))
        if 1 / multi_freq[i] < min_multiplicator:
            break

оплата 192.1899
хамство 153.7519
оскорблять 153.7519
осадок 96.0949
грамм 76.876
продемонстрировать 76.876
недовольный 51.2506
посчитать 51.2506
бесполезный 38.438
обидно 30.7504
предупреждать 26.9066
увы 25.6253
исключать 25.6253
привязанность 25.6253
вероника 21.9646
платный 19.9308
оплачивать 19.219
резко 19.219
id 19.219
обесценивать 17.0835
неприятный 16.6218
отрицательный 16.0158
свидание 15.3752
разочарование 14.9481
условие 14.2363
извинять 14.1614
опускать 13.9774
деньги 13.7617
тратить 13.3697
браться 12.8127
бабушка 12.8127
прекращать 12.8127
отказываться 11.8271
возвращать 11.8271
тип 11.3053
бежать 10.9823
обижаться 10.9823
сожаление 10.4332
видимо 9.8841
цена 9.0442
брак 9.0442
сутки 8.9689
заявлять 8.5418
нормально 8.5418
попросить 8.2367
диагноз 8.0922
образование 7.9527
завершение 7.6876
виноватый 7.3215
должно 7.3215
тест 7.3215
либо 7.2071
никакой 7.0905
бесплатный 7.0807
состояться 6.9887
занимать 6.9887
соглашаться 6.9188
далее 6.8639
пустой 6.6849
тянуть 6.6849
пр

In [52]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(multi_freq[:50])
    for i in multi_freq[-200:].index:
#         print(neg[i] / neg[i].min())
        if neg[i] / neg.min() > 4:
            print(i, multi_freq[i].round(4))

общаться 1.8645
разбираться 1.91
помогать 1.9476
решение 1.9772
важный 1.9805
увидеть 2.0206
ситуация 2.0813
полезный 2.1281
оксана 2.2218
каждый 2.233
помощь 2.2576
чувствовать 2.2605
смочь 2.2699
ваш 2.3109
действительно 2.3154
хороший 2.3507
появляться 2.3622
понимание 2.4758
именно 2.5273
очень 2.5305
мысль 2.7403
татьяна 2.919
новый 3.8347
поддержка 4.1018
жизнь 4.5988
большой 9.0089
спасибо 19.0957


In [53]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(multi_freq[-30:])

спокойно          6.217809
проделывать       6.530000
светлана          6.686095
изменение         6.738127
жизненный         6.764143
людмила           6.842191
многое            6.998287
уверенность       7.050319
андрей            7.466574
владимировна      7.544621
ценный            7.570637
сергеевич         7.934860
удаваться         8.169004
сергей            9.001514
большой           9.008947
замечательный     9.235657
грамотный         9.469801
внимательный     10.172231
легко            10.276295
сила             10.380358
главное          10.640518
огромный         11.791723
благодарить      12.799840
профессионал     12.851872
наталья          13.346175
александр        14.542908
благодаря        17.248565
спасибо          19.095697
благодарность    20.578605
благодарный      27.915099
dtype: float64


In [54]:
rev_neg = pd.Series(reviews_processed)[df[df.columns[1]] == 'о']
rev_pos = pd.Series(reviews_processed)[df[df.columns[1]] == 'п']

In [55]:
for i in rev_neg[rev_neg.apply(lambda x: 'н' in x)]:
    print(' '.join(i))

In [61]:
rev_neg[rev_neg.apply(lambda x: 'свидание' in x)]

4388    [ходить, алена, викторовна, очный, консультаци...
4568    [замешательство, видеть, данный, специалист, в...
6144    [позитиффффф, подход, часть, эмоция, просто, о...
dtype: object

In [58]:
# df.iloc[np.where(rev_neg.apply(lambda x: 'грамм' in x))[0], 'отзыв']

In [ ]:
df['review_lem'] = [' '.join(i) for i in reviews_processed]

In [ ]:
df['review_lem']

In [ ]:
print(count_ngrams(df.loc[df[df.columns[1]] == 'п', 'review_lem'], 2).index[:50])

In [ ]:
reviews_processed[0]

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [ ]:
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

In [ ]:
lda = LdaModel(common_corpus, num_topics=10)

In [ ]:
lda.show_topics()

In [ ]:
def elems_to_text(lists):
    return [' '.join(i) for i in lists]

In [ ]:
reviews_proc_text = elems_to_text(reviews_processed)

In [ ]:
all_words = []
for i in reviews_proc_text:
    all_words.extend(word_tokenize(i))

In [ ]:
pd.Series(all_words).value_counts()[:20].index

In [ ]:
def plot_wordcloud(text, mask=None, max_words=400, max_font_size=120, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown', 'это'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='white',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    mask = mask)
    wordcloud.generate(text)
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'green', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    plt.savefig('hello.png', dpi=1000)
d = './masks/'

In [ ]:
comments_text = ' '.join(all_words)
comments_mask = np.array(Image.open(d + 'psy.png'))
plot_wordcloud(comments_text, comments_mask, max_words=400, max_font_size=120, 
               title = 'Most common words in all of the comments', title_size=50)

In [ ]:
num_words = 10000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [ ]:
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(reviews_proc_text)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
seq = tokenizer.texts_to_sequences(reviews_proc_text)

In [ ]:
max_len = max([len(x) for x in seq])

In [ ]:
max_len

In [ ]:
# df['отзыв'].sort_values(key=lambda x: x.str.len(), ascending=False).reset_index(drop=True)[0]
# самый длинный отзыв по элементам

In [ ]:
seq_padded = pad_sequences(seq, maxlen=max_len, padding=pad_type, truncating=trunc_type)

In [ ]:
seq_padded.shape

In [ ]:
# print("Word index:\n", word_index)
# print("\nTraining sequences:\n", train_sequences)
# print("\nPadded training sequences:\n", train_padded)
# print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(seq))
print("Padded Training sequences data type:", type(seq_padded))

In [ ]:
tokenizer.sequences_to_texts(seq)[:5]

In [ ]:
df['отзыв'].isna().sum()

In [ ]:
all_reviews = df['отзыв'].copy()
left_reviews = all_reviews.copy()
left_indices = list(left_reviews.index)
labels = {}

In [ ]:
for i in random.choices(left_indices, k=3):
    print(left_reviews[i])
    left_indices.remove(i)
    label = input('label it please')
    labels[i] = label

In [ ]:
# labels

In [ ]:
len(left_indices)

In [ ]:
# labels

In [ ]:
seq_padded.shape

In [ ]:
# list(labels.keys())

In [ ]:
pd.get_dummies(list(labels.values())).to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X = np.asarray(seq_padded[list(labels.keys()), :])
y = np.asarray(pd.get_dummies(list(labels.values())).to_numpy())

In [ ]:
embedding_vector_length = 32
model = Sequential()
emb = layers.Embedding(num_words, embedding_vector_length, input_length=max_len)
model.add(emb)
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='elu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(16, activation='elu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(3, activation='softmax'))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X, y)

In [ ]:
probas = model.predict_proba(seq_padded)

In [ ]:
plt.hist(probas[:, 0])